In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [1]:
# 修改參數
!sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/g" darknet/Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile

'sed' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'sed' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'sed' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'sed' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [2]:
# 參數
!head darknet/Makefile

'head' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [3]:
!cd darknet; make

�t�Χ䤣����w�����|�C


In [4]:
# 建立資料夾
import os
import shutil
if not os.path.exists('waymo_object_detection'):
 os.mkdir('waymo_object_detection')
if not os.path.exists('waymo_object_detection/cfg'):
 os.mkdir('waymo_object_detection/cfg') 
 os.mkdir('waymo_object_detection/weights')
if not os.path.exists('waymo_object_detection/cfg/face.data'):
 shutil.copyfile('darknet/cfg/coco.data', 'waymo_object_detection/cfg/object.data')
if not os.path.exists('waymo_object_detection/cfg/face.names'):
 shutil.copyfile('darknet/cfg/coco.names', 'waymo_object_detection/cfg/object.names')

In [ ]:
import matplotlib.pyplot as plt
HEIGHT, WIDTH = (640, 960)

folder_path = {
    'train':'./train',
    'val':'./valid'
}

for key in folder_path.keys():
    if not os.path.exists(folder_path[key]):
        os.mkdir(folder_path[key])

In [ ]:
def box2yoloFormat(center_x, center_y, length, width, ORIGIN_WIDTH, ORIGIN_HEIGHT):
    box_center_x = center_x / ORIGIN_WIDTH
    box_center_y = center_y / ORIGIN_HEIGHT
    box_width = length / ORIGIN_WIDTH
    box_height = width / ORIGIN_HEIGHT
    return (box_center_x, box_center_y, box_width, box_height)

def imageDecode(image):
    image = tf.image.decode_jpeg(image)   
    h, w = image.shape[:2]
    image = tf.image.resize(image, (HEIGHT, WIDTH)) 
    images = tf.expand_dims(image, axis=0) / 255.0
    return images, h, w

In [ ]:
'''
enum Name {
    UNKNOWN = 0;
    FRONT = 1;
    FRONT_LEFT = 2;
    FRONT_RIGHT = 3;
    SIDE_LEFT = 4;
    SIDE_RIGHT = 5;
  }
'''
def createImgLabel(dataSetType):
    # WRITE train.txt
    with open(f'/content/waymo_object_detection/cfg/{dataSetType}.txt', 'w') as out:
        for img in [f for f in os.listdir(f'/content/waymo_object_detection/{dataSetType}') if f.endswith('jpg')]:
            out.write(f'/content/waymo_object_detection/{dataSetType}/' + img + '\n')
    

In [ ]:
createImgLabel('train')

In [ ]:
createImgLabel('val')

In [ ]:
classes = [
    'TYPE_UNKNOWN',
    'TYPE_VEHICLE',
    'TYPE_PEDESTRIAN',
    'TYPE_SIGN',
    'TYPE_CYCLIST'
]

In [ ]:
with open('/content/waymo_object_detection/cfg/object.names',mode='w') as f:   
    for c in classes:
        f.write(c)
        f.write('\n')

In [ ]:
with open('/content/waymo_object_detection/cfg/object.data',mode='w') as f:   
    f.write(f'classes = {len(classes)}\n')
    f.write(f'train = /content/waymo_object_detection/cfg/train.txt\n')
    f.write(f'valid = /content/waymo_object_detection/cfg/val.txt\n')
    f.write(f'names = /content/waymo_object_detection/cfg/object.names\n')
    f.write(f'backup = /content/waymo_object_detection/cfg/weights\n')

In [ ]:
!cp ./darknet/cfg/yolov4-tiny.cfg /content/waymo_object_detection/cfg/yolo-obj.cfg

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
!mv ./yolov4-tiny.conv.29 ./waymo_object_detection/cfg

In [ ]:
!cd ./darknet; ./darknet detector train /content/waymo_object_detection/cfg/object.data /content/waymo_object_detection/cfg/yolo-obj.cfg /content/waymo_object_detection/cfg/yolov4-tiny.conv.29 -dont_show -map

In [ ]:
!cd ./darknet; ./darknet detector test /content/waymo_object_detection/cfg/object.data /content/waymo_object_detection/cfg/yolo-obj.cfg /content/waymo_object_detection/cfg/weights/yolo-obj_last.weights /content/waymo_object_detection/train/0.jpg

In [ ]:
!cd ./darknet; ./darknet detector map /content/waymo_object_detection/cfg/object.data /content/waymo_object_detection/cfg/yolo-obj.cfg /content/waymo_object_detection/cfg/weights/yolo-obj_best.weights